In [1]:
print('Paskaita nr. 50, 2024-11-04 „Unittest“')
print('Mano  darbai: entitiesWithProperties Class')
print('Kol kas čia ir def Class, ir klasės panaudojimo bandymai.')
print('Kai veiks, pervadinsiu į *.py ir importuosiu klasę.')

import sqlite3
import datetime as dt

class entitiesWithProperties:
    # Parameters of the system, not of a particular data set:
    production=False # 'True' for production, 'False' for testing.
    class_type='entitiesWithProperties'

    # Parameters of the entity:
    _entity_id=None

    def __init__(self,name): # constructor.
        self._name=name
        self._connection = sqlite3.connect(name)
        self._cursor = self._connection.cursor()
        if not entitiesWithProperties.production:
            print(f"Connected '{name}'\n")

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if not entitiesWithProperties.production:
            print(f"{self.class_type}.__exit__()")
        self.close() # .__exit__() vykdomas jei nebelieka kintamojo
    
    def __str__(self):
        rez=f'The structure of database "{self._name}"\n'
        for (tableName,) in self.query(
        "select NAME from SQLITE_MASTER where TYPE='table' order by NAME;"
        ):
            rez+="{}:\n".format(tableName)
            for ( columnID,columnName,columnType,columnNotNull,columnDefault,columnPK,
            ) in self.query("pragma table_info('{}');".format(tableName)):
                rez+="  {id}: {name}({type}){null}{default}{pk}\n".format(
                    id=columnID,
                    name=columnName,
                    type=columnType,
                    null=" not null" if columnNotNull else "",
                    default=" [{}]".format(columnDefault) if columnDefault else "",
                    pk=" *{}".format(columnPK) if columnPK else "",
                )
        return rez

    @property
    def connection(self):
        return self._connection

    @property
    def cursor(self):
        return self._cursor

    def commit(self):
        self.connection.commit()

    def close(self, commit=True):
        if not entitiesWithProperties.production:
            print(f"{self.class_type}.close()")
        commit and self.commit()
        self.connection.close()

    def execute(self, sql, params=None): # result with .fetchall() or .fetchone()
        return self._cursor.execute(sql, params or ())


    def selectentity(self, sql='', params=''):
        return self.execute("select * from entity "+sql, params or ())
        # order by entity_id limit 10 where ...

    def nextentity(self):
        rez=self._cursor.fetchone()
        if rez:
            self._entity_id=rez[0]
            return rez
        else:
            self._entity_id=None
            return None


    def fetchall(self):
        return self._cursor.fetchall()

    def fetchone(self):
        return self._cursor.fetchone()

    def query(self, sql, params=None):
        self._cursor.execute(sql, params or ())
        return self.fetchall()


    def setentityid(self,entity_id):
        self._entity_id=entity_id

    def resetentityid(self):
        self._entity_id=None

    def getentityid(self):
        return self._entity_id

    def setproperty(self,property_name,property_value,entity_id=None):
        if entity_id:
            self._entity_id=entity_id
        if not property_value:
            property_value=1
        dabar=int(dt.datetime.today().timestamp()) # now(), today()
        if self._entity_id and property_name:
            sql_template='''INSERT OR REPLACE into entity_property 
                (entity_id,property_name,property_value,create_timestamp,update_timestamp)
                values (?,?,?,?,?)'''
            self.execute(sql_template,(self._entity_id,property_name,property_value,dabar,dabar))

if not entitiesWithProperties.production:
    print("Class 'entitiesWithProperties()' defined.")

Paskaita nr. 50, 2024-11-04 „Unittest“
Mano  darbai: entitiesWithProperties Class
Kol kas čia ir def Class, ir klasės panaudojimo bandymai.
Kai veiks, pervadinsiu į *.py ir importuosiu klasę.
Class 'entitiesWithProperties()' defined.


In [2]:
db_file_name='auto_skelbimai_txt.db'
db_connect = sqlite3.connect(db_file_name)
cs = db_connect.cursor()
print(f"sqlite3.connect('{db_file_name}')")
print(dt.datetime.now().replace(microsecond=0))

db_objektas=entitiesWithProperties(db_file_name)

db_objektas.selectentity("limit 100")

for rez in db_objektas.nextentity() :
    id=db_objektas.getentityid()
    print('Next:',id,)
    db_objektas.setproperty('entity_id',id)

# for i in range(5):
#     print(db_objektas.fetchone())

# print(db_objektas)
db_objektas.close()

sqlite3.connect('auto_skelbimai_txt.db')
2024-11-04 23:16:23
Connected 'auto_skelbimai_txt.db'

Next: 18267894
Next: 18267894
Next: 18267894
Next: 18267894
entitiesWithProperties.close()
